In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import boto3
import json

In [2]:
Target_year = 2023

pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.1f}'.format

User_Name = 'dsadmin'


AWS_folder = "./"

src_folder = os.path.join(AWS_folder,'Downloaded',User_Name)
out_folder = os.path.join(AWS_folder,'ToUpload',User_Name)

In [52]:
# Create Out folder if not exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

base.json file이 모든 정보를 가지고 있으므로 이화일만 다운로드
그 후 내용 화일로 분리

In [6]:
s3 = boto3.client('s3')
content_object = s3.get_object(Bucket = 'dsmapdata', Key = User_Name + '/base.json')
file_content = content_object['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

In [7]:
json_content["label_info"]

[{'L1': '건강한잔디', 'L2': '일반'},
 {'L1': '건강한잔디', 'L2': '배토'},
 {'L1': '건강한잔디', 'L2': '통기 (유공)'},
 {'L1': '건강한잔디', 'L2': '통기 (무공)'},
 {'L1': '건강한잔디', 'L2': '시비 (알비)'},
 {'L1': '건강한잔디', 'L2': '시비 (액비)'},
 {'L1': '건강한잔디', 'L2': '깎기'},
 {'L1': '건강한잔디', 'L2': '보식'},
 {'L1': '건강한잔디', 'L2': '농약살포'},
 {'L1': '건강한잔디', 'L2': '제초제 (발아전)'},
 {'L1': '건강한잔디', 'L2': '제초제 (경엽)'},
 {'L1': '건강한잔디', 'L2': '버티칼모잉'},
 {'L1': '건강한잔디', 'L2': '브러쉬작업'},
 {'L1': '건강한잔디', 'L2': '스위퍼작업'},
 {'L1': '건강한잔디', 'L2': '구르밍작업'},
 {'L1': '건강한잔디', 'L2': '기타'},
 {'L1': '물리적피해', 'L2': '볼마크'},
 {'L1': '물리적피해', 'L2': '건조피해'},
 {'L1': '물리적피해', 'L2': '플레이어 답압'},
 {'L1': '물리적피해', 'L2': '고온피해'},
 {'L1': '물리적피해', 'L2': '디봇'},
 {'L1': '물리적피해', 'L2': '과습피해'},
 {'L1': '물리적피해', 'L2': '약해'},
 {'L1': '물리적피해', 'L2': '음지피해'},
 {'L1': '물리적피해', 'L2': '장비피해'},
 {'L1': '물리적피해', 'L2': '저온피해'},
 {'L1': '물리적피해', 'L2': '기타피해'},
 {'L1': '물리적피해', 'L2': '나방류'},
 {'L1': '물리적피해', 'L2': '굼벵이류'},
 {'L1': '물리적피해', 'L2': '응애'},
 {'L1': '물리적피해', 'L2': '기타'},


In [54]:
for key_ in json_content.keys():
  if 'user' not in key_ :
    file_to_save = os.path.join(src_folder ,'{}.json'.format(key_))
    print(file_to_save)
    with open(file_to_save, "w", encoding="utf-8") as file:
      file.write(json.dumps(json_content[key_], ensure_ascii=False))

./Downloaded\dsadmin\area_def.json
./Downloaded\dsadmin\course_info.json


작업.

In [ ]:

import shutil

files = glob.glob(os.path.join(src_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]

for f_ in target_files:
  shutil.copy(f_, out_folder)

작업이 완료되면 UPload folder로 이동

In [1]:
files = glob.glob(os.path.join(out_folder,'*.json'))
target_files = [ x for x in files if ('~$' not in x) ]
target_files

NameError: name 'glob' is not defined

각 화일을 base.json으로 통합, 그후 가가 화일과 통합화일을 Upload

In [15]:
Total_json = {}

s3 = boto3.client('s3')

for _file in target_files:
  f_ = os.path.splitext(os.path.basename(_file))[0]
  json_content_ = json.load(open(_file, encoding="utf8"))
  Total_json[f_] = json_content_
  s3.put_object(Body=json.dumps(json_content_).encode('utf-8'),Bucket='dsmapdata',Key='baseinfo/{}.json'.format(f_))
  
s3.put_object(
  Body=json.dumps(Total_json).encode('utf-8'),
  Bucket='dsmapdata',
  Key=User_Name + '/base.json'
  )

{'ResponseMetadata': {'RequestId': '7ET2YM8RZN2CDBA0',
  'HostId': 'WkJDwC+zOiJBxTVVAYF8mKsTSokpK8WH2aZ5RMGDFWs45yk3zbRm+ltVN6iueJgTZ5TG4bkNTtQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'WkJDwC+zOiJBxTVVAYF8mKsTSokpK8WH2aZ5RMGDFWs45yk3zbRm+ltVN6iueJgTZ5TG4bkNTtQ=',
   'x-amz-request-id': '7ET2YM8RZN2CDBA0',
   'date': 'Thu, 14 Sep 2023 15:09:12 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7de37b86c2344bd128d8bf283132c9d5"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7de37b86c2344bd128d8bf283132c9d5"',
 'ServerSideEncryption': 'AES256'}

In [16]:
Total_json.keys()

dict_keys(['area_def', 'course_info', 'label_info'])

In [17]:
Total_json['label_info']

[{'L1': '건강한잔디', 'L2': '일반'},
 {'L1': '건강한잔디', 'L2': '배토'},
 {'L1': '건강한잔디', 'L2': '통기 (유공)'},
 {'L1': '건강한잔디', 'L2': '통기 (무공)'},
 {'L1': '건강한잔디', 'L2': '시비 (알비)'},
 {'L1': '건강한잔디', 'L2': '시비 (액비)'},
 {'L1': '건강한잔디', 'L2': '깎기'},
 {'L1': '건강한잔디', 'L2': '보식'},
 {'L1': '건강한잔디', 'L2': '농약살포'},
 {'L1': '건강한잔디', 'L2': '제초제 (발아전)'},
 {'L1': '건강한잔디', 'L2': '제초제 (경엽)'},
 {'L1': '건강한잔디', 'L2': '버티칼모잉'},
 {'L1': '건강한잔디', 'L2': '브러쉬작업'},
 {'L1': '건강한잔디', 'L2': '스위퍼작업'},
 {'L1': '건강한잔디', 'L2': '구르밍작업'},
 {'L1': '건강한잔디', 'L2': '기타'},
 {'L1': '물리적피해', 'L2': '볼마크'},
 {'L1': '물리적피해', 'L2': '건조피해'},
 {'L1': '물리적피해', 'L2': '플레이어 답압'},
 {'L1': '물리적피해', 'L2': '고온피해'},
 {'L1': '물리적피해', 'L2': '디봇'},
 {'L1': '물리적피해', 'L2': '과습피해'},
 {'L1': '물리적피해', 'L2': '약해'},
 {'L1': '물리적피해', 'L2': '음지피해'},
 {'L1': '물리적피해', 'L2': '장비피해'},
 {'L1': '물리적피해', 'L2': '저온피해'},
 {'L1': '물리적피해', 'L2': '기타피해'},
 {'L1': '물리적피해', 'L2': '나방류'},
 {'L1': '물리적피해', 'L2': '굼벵이류'},
 {'L1': '물리적피해', 'L2': '응애'},
 {'L1': '물리적피해', 'L2': '기타'},
